# Monitorización de Sensores con Cassandra

Este cuaderno muestra cómo conectar a una base de datos Cassandra desde Python, definir un modelo de datos para almacenar lecturas de sensores, insertar datos de ejemplo y realizar consultas. Además, se utilizarán UDTs (User-Defined Types) y se configurarán TTLs (Time-To-Live) para algunas entradas.

In [130]:
from cassandra.cluster import Cluster
from cassandra.query import SimpleStatement
from datetime import datetime
import uuid

## Conexión a Cassandra

Primero, conectamos a nuestro clúster de Cassandra. Asegúrate de que el clúster está corriendo y accesible desde tu máquina.

In [131]:
# Conectar a Cassandra
cluster = Cluster(['127.0.0.1'])  # Sustituye con la dirección de tu clúster
session = cluster.connect()

## Crear un Keyspace

Un keyspace en Cassandra es un contenedor para las tablas. Aquí creamos un keyspace llamado `sensor_data`.

In [132]:
# Crear un keyspace
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sensor_data
    WITH replication = {'class': 'SimpleStrategy', 'replication_factor': '3'}
""")

## Seleccionar el Keyspace

Después de crear el keyspace, lo seleccionamos para que las siguientes operaciones se apliquen dentro de este contexto.

In [133]:
# Seleccionar el keyspace
session.set_keyspace('sensor_data')

## Crear Tipos de Datos Definidos por el Usuario (UDTs)

Los UDTs nos permiten definir tipos de datos personalizados que pueden ser reutilizados en nuestras tablas. Aquí creamos dos UDTs: `location` y `sensor_info`.

In [134]:
# Crear UDT para la ubicación
session.execute("""
    CREATE TYPE IF NOT EXISTS location (
        latitude float,
        longitude float
    )
""")

## Crear la Tabla

Creamos una tabla llamada `temperature_readings` que usa los UDTs para almacenar información detallada sobre los sensores y sus lecturas de temperatura. Usamos `sensor_id` como clave primaria.

In [135]:
# Crear una tabla para almacenar las lecturas de temperatura
session.execute("""
    CREATE TABLE IF NOT EXISTS temperature_readings (
        sensor_id UUID,
        purpose text,
        location frozen<location>,
        timestamp TIMESTAMP,
        temperature FLOAT,
        PRIMARY KEY (sensor_id, timestamp)
    ) WITH CLUSTERING ORDER BY (timestamp DESC)
""")

## Insertar Datos de Ejemplo

Insertamos datos de ejemplo en la tabla `temperature_readings`, incluyendo un TTL (Time-To-Live) de 60 segundos para una de las entradas. Esto significa que la entrada será eliminada automáticamente después de 60 segundos.

In [136]:
# Registrar UDTs en el mapeador del cluster
cluster.register_user_type('sensor_data', 'location', dict)

# Insertar datos de ejemplo
sensor_id = uuid.uuid4()
location_data = {
    'latitude': 40.7128,
    'longitude': -74.0060
}
sensor_data = {
    'purpose': 'temperature monitoring',
    'location': location_data
}

# Insertar con TTL de 60 segundos
session.execute("""
    INSERT INTO temperature_readings (sensor_id, purpose, location, timestamp, temperature)
    VALUES (%s, %s, %s, %s, %s) USING TTL 60
    """,
    (sensor_id, sensor_data['purpose'], sensor_data['location'], datetime.utcnow(), 22.5)
)

# Insertar sin TTL
session.execute("""
    INSERT INTO temperature_readings (sensor_id, purpose, location, timestamp, temperature)
    VALUES (%s, %s, %s, %s, %s)
    """,
    (sensor_id, sensor_data['purpose'], sensor_data['location'], datetime.utcnow(), 23.0)
)

## Consultar Lecturas por Propósito

Definimos una función para obtener lecturas de temperatura filtradas por el propósito del sensor. Utilizamos `ALLOW FILTERING` en la consulta para permitir el filtrado por una columna que no está en la clave primaria.

In [137]:
# Función para obtener lecturas de temperatura por propósito
def get_temperature_readings_by_purpose(purpose):
    query = SimpleStatement("""
        SELECT sensor_id, purpose, location, timestamp, temperature FROM temperature_readings
        WHERE purpose = %s ALLOW FILTERING
    """, fetch_size=10)
    rows = session.execute(query, (purpose,))
    for row in rows:
        print(f"Sensor ID: {row.sensor_id}, Purpose: {row.purpose}, "
              f"Location: ({row.location['latitude']}, {row.location['longitude']}), "
              f"Timestamp: {row.timestamp}, Temperature: {row.temperature}")

## Obtener y Mostrar Lecturas

Usamos la función `get_temperature_readings_by_purpose` para obtener y mostrar las lecturas de temperatura para los sensores cuyo propósito es `temperature monitoring`.

In [138]:
# Obtener las lecturas de temperatura para sensores con el propósito de 'temperature monitoring'
get_temperature_readings_by_purpose('temperature monitoring')

Sensor ID: cbb82f31-915c-4845-9f1e-5762f753462f, Purpose: temperature monitoring, Location: (None, None), Timestamp: 2024-05-31 15:24:34.410000, Temperature: 23.0
Sensor ID: cbb82f31-915c-4845-9f1e-5762f753462f, Purpose: temperature monitoring, Location: (None, None), Timestamp: 2024-05-31 15:24:34.392000, Temperature: 22.5


## Cerrar la Conexión

Finalmente, cerramos la conexión al clúster de Cassandra.

In [36]:
# Cerrar la conexión
cluster.shutdown()